# Comparison Titanic models
In this notebook, we will compare the three models we've created for the Titanic dataset.

## Imports

In [81]:
import io, boto3
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pycaret.classification import *
from sklearn.metrics import confusion_matrix, roc_auc_score

## Load data
Since we will be making predictions in this file, we will load the test files.

In [82]:
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

We will also load the models.

In [83]:
pycaret_model = load_model('pycaret_best_pipeline')
    
with open('knn_model.pkl', 'rb') as file:
    knn_model = pickle.load(file)

Transformation Pipeline and Model Successfully Loaded


Since we saved the predictions from the AWS model, we will need to load them as well.

In [84]:
bucket='titanic-bucket-mj'
prefix='titanic'

In [85]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/titanic_test.csv.out".format(prefix))
aws_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=',',names=['survived'])

In [86]:
def convert_to_label(x):
    threshold = 0.7
    return 1 if x > threshold else 0

aws_predicted = aws_predicted['survived'].apply(convert_to_label)

aws_predicted.head()

0    0
1    0
2    0
3    1
4    1
Name: survived, dtype: int64

## Predict
Here, we use each model individually to predict the results it generates.

In [87]:
knn_predicted = knn_model.predict(x_test)
print(knn_predicted)

[0 0 0 ... 0 1 1]


In [88]:
pycaret_predicted = pycaret_model.predict(x_test)
print(pycaret_predicted)

[0 0 0 ... 0 1 1]


## Compare
In this section, we will use various metrics to compare the predictions made by our models.

In [92]:
p_TN, p_FP, p_FN, p_TP = confusion_matrix(y_test, pycaret_predicted).ravel()
a_TN, a_FP, a_FN, a_TP = confusion_matrix(y_test, aws_predicted).ravel()
k_TN, k_FP, k_FN, k_TP = confusion_matrix(y_test, knn_predicted).ravel()

data_classification = {
    'Model': ['pycaret', 'aws', 'knn'],
    'TN': [p_TN, a_TN, k_TN],
    'FP': [p_FP, a_FP, k_FP],
    'FN': [p_FN, a_FN, k_FN],
    'TP': [p_TP, a_TP, k_TP]
}

confusion_matrix_df = pd.DataFrame(data_classification)

models = confusion_matrix_df['Model']
confusion_matrices = [
    np.array([[row['TN'], row['FP']],
              [row['FN'], row['TP']]]) for _, row in confusion_matrix_df.iterrows()
]

# Create a heatmap for each model
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

for ax, matrix, model in zip(axes, confusion_matrices, models):
    sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
    ax.set_title(f'{model} Confusion Matrix')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    ax.set_xticklabels(['Died', 'Survived'])
    ax.set_yticklabels(['Died', 'Survived'])

#since we have some font-errors that won't be resolved that easily within sagemaker, so we save the matrices as an image
plt.savefig('confusion_matrices.png')
print("Plot saved as 'confusion_matrices.png'.")
    
# Display the plots
plt.show()

Due to an unexpected error with fonts in SageMaker, we will display an image of the results instead. As seen, it is relatively easy for most models to predict when a passenger died, but determining survival is much more challenging. This could be attributed to the imbalance in the 'survived' column, with fewer 'survived' values compared to 'died' values.

![confusion matrices](images/confusion_matrices.png)

In [ ]:
def calculate_metrics(row):
    TN, FP, FN, TP = row['TN'], row['FP'], row['FN'], row['TP']
    total = TN + FP + FN + TP
    
    accuracy = (TP + TN) / total
    sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
    specificity = TN / (TN + FP) if TN + FP > 0 else 0
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    npv = TN / (TN + FN) if TN + FN > 0 else 0
    fpr = FP / (FP + TN) if FP + TN > 0 else 0
    fnr = FN / (FN + TP) if FN + TP > 0 else 0
    fdr = FP / (FP + TP) if FP + TP > 0 else 0
    
    return pd.Series([accuracy, sensitivity, specificity, precision, npv, fpr, fnr, fdr])

metrics_df = confusion_matrix_df.apply(calculate_metrics, axis=1)
metrics_df.columns = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision',
                      'Negative Predictive Value', 'False Positive Rate',
                      'False Negative Rate', 'False Discovery Rate']

# Has to be added afterwards since it uses the predicted values
validation_auc = [
    roc_auc_score(y_test, pycaret_predicted),
    roc_auc_score(y_test, aws_predicted),
    roc_auc_score(y_test, knn_predicted)
]

metrics_df['Validation AUC'] = validation_auc

# We add the Model-column since that makes it clearer which metric responds to which model
metrics_df.insert(0, 'Model', confusion_matrix_df['Model'])

display(metrics_df)

,Model,Accuracy,Sensitivity,Specificity,Precision,Negative Predictive Value,False Positive Rate,False Negative Rate,False Discovery Rate,Validation AUC
0,pycaret,0.879929,0.759241,0.936632,0.849153,0.892244,0.063368,0.240759,0.150847,0.847937
1,aws,0.864172,0.637747,0.970553,0.910518,0.850802,0.029447,0.362253,0.089482,0.804150
2,knn,0.864826,0.750951,0.918328,0.812028,0.886983,0.081672,0.249049,0.187972,0.834639


We can conclude that all models are highly accurate. However, while the PyCaret and KNN models are better at correctly predicting positives, the AWS/XGBoost model performs significantly better at predicting negatives compared to the others.